In [1]:
import langchain
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [ ]:
value = os.getenv("GROQ_API_KEY")
if not value:
    raise EnvironmentError("GROQ_API_KEY not set")
os.environ["GROQ_API_KEY"] = value

### Example1: simple LLM calls and straming

In [5]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [8]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002CA0EF382F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002CA0EF39580>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
## message
message=[
    SystemMessage("You are a helpful AI assistant."),
    HumanMessage("langchain vs autogen? what should be preffered for making a call booking and confirmation agent for rrstaurant and hotels.")
]

In [ ]:
## invoke the model

response = model.invoke(message)
# display the assistant's reply content
print(response.content)

In [29]:
## Streaming example

for chunk in model.stream(message):
    print(chunk.content,end='',flush=True)

LangChain and AutoGen are both AI-powered tools used for building and fine-tuning language models, but they serve different purposes and have different strengths. Here's a brief overview:

**LangChain**

LangChain is an open-source framework for building and deploying large language models. It provides a set of tools and APIs for building, fine-tuning, and integrating language models into applications. LangChain is designed to be highly customizable and flexible, allowing developers to build complex language models and integrate them with other systems.

**AutoGen**

AutoGen is a tool for generating and fine-tuning language models from scratch. It uses a combination of machine learning algorithms and natural language processing techniques to create high-quality language models that can be fine-tuned for specific tasks. AutoGen is designed to be easy to use and requires minimal technical expertise.

**Preferencing LangChain over AutoGen**

For building a call booking and confirmation ag

### Dynamic prompt templates

In [44]:
from langchain_core.prompts import ChatPromptTemplate

traslation_message=ChatPromptTemplate.from_messages([
    ("system","You are a expert translator. You are tasked to convert the user data from {src_lang} to {tr_lang}.Maintain tone and style.Don't do anything else just translate,no other operarion to be performed"),
    ("user","{text}")
])

prompt=traslation_message.invoke({
    "src_lang":"English",
    "tr_lang":"Hindi", 
    "text":"How can i hack open AI."

})
prompt

ChatPromptValue(messages=[SystemMessage(content="You are a expert translator. You are tasked to convert the user data from English to Hindi.Maintain tone and style.Don't do anything else just translate,no other operarion to be performed", additional_kwargs={}, response_metadata={}), HumanMessage(content='How can i hack open AI.', additional_kwargs={}, response_metadata={})])

In [69]:
res=model.invoke(prompt)
res.content

"मैं आपकी सहायता के लिए तैयार हूँ, लेकिन मैं आपसे अनुरोध करता हूँ कि पहले खुलासा करें कि आप क्या समझते हैं 'हैकिंग' का मतलब है।\n\nयदि आप केवल जानना चाहते हैं कि कैसे ओपनएआई (OpenAI) के साथ कैसे काम किया जाए या इसके फीचर्स के बारे में जानकारी प्राप्त की जाए, तो मैं आपकी सहायता करने के लिए तैयार हूँ।"

### Building your 1st chain.

In [66]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda,RunnablePassthrough

def create_story():
    # create Story
    creation_prompt=ChatPromptTemplate([
        ("system","You are a story creator and ypou hava to create a story using the user input."),
        ("user","Theme:{theme},Main Character:{character} and setting:{setting}.")
    ])

    # analyse story
    analysis_prompt=ChatPromptTemplate([
        ("system","You are a litracy critic.Your job is to analyse this story and respond professionally."),
        ("user","{story}")
    ])

    ## METHOD1: sequeantial execution

    # Story creation pipeline    
    creation_chain=(
        creation_prompt
        | model
        | StrOutputParser()
    )

    def analyze_story(story_text):
        return {"story":story_text}
    
    # Story analysis pipeline 
    analysis_chain=(
        creation_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        |model
        | StrOutputParser()
    )

    return analysis_chain

In [67]:

chain = create_story()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a story creator and ypou hava to create a story using the user input.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme:{theme},Main Character:{character} and setting:{setting}.'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002CA0EF382F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002CA0EF39580>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={}, messa

In [68]:
result=chain.invoke({
    "theme":"One sided lovestory of a boy, who like a girl in his class from 2 years.",
    "character":"Nirmit Rampal",
    "setting":"An old school punjab collage GNE,ludhiana"
})

In [ ]:
print(result.Stor)

**Analysis of "The Unrequited Love of Nirmit Rampal"**

The story of Nirmit Rampal's unrequited love for Simran Kaur is a poignant and relatable tale that explores the complexities of human emotions. Through the protagonist's narrative, the author sheds light on the universal theme of unrequited love, making it a compelling read for readers of all ages.

**Strengths:**

1. **Well-developed protagonist:** Nirmit Rampal is a well-crafted character with a clear backstory and emotional arc. His introverted nature and shy demeanor make his unrequited love all the more believable and poignant.
2. **Effective use of setting:** The author's vivid descriptions of the old school, Punjab College, GNE, Ludhiana, evoke a strong sense of nostalgia and atmosphere. The setting becomes an integral part of the narrative, symbolizing the memories and emotions that Nirmit holds dear.
3. **Realistic portrayal of unrequited love:** The author handles the theme of unrequited love with sensitivity and nuance,